# Figures Ecozone and graphical abstract

In [ ]:
# Libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle

In [ ]:
# Directories
dir_data = '../data/'
dir_nearth = '../data/naturalearth/'
dir05 = '../paper_deficit/output/05_prep_other/'
dir06 = '../paper_deficit/output/06_eval/'

---

In [ ]:
# Get prepared primary and secondary data
def prep_map_data(scen):
    
    """Get and prepare prepared csv of ecozone_map_data"""
    
    # Get map data
    df_map = pd.read_csv(
        os.path.join(dir05, f'fig_ecozone/df_ecozone_map_data_{scen}.csv'))

    # Prepare map data
    df_map_world = df_map \
        .drop('cat0', axis=1) \
        .sum(axis=0) \
        .to_frame(name='values') \
        .reset_index(names='index') \
        .assign(a = 0) \
        .pivot(index='a', columns='index', values='values') \
        .reset_index(drop=True) \
        .round(0) \
        .astype('int') \
        .assign(cat0='world')

    return pd.concat([df_map_world, df_map])


df_map_data_prim = prep_map_data('prim')
df_map_data_secd = prep_map_data('secd')

In [ ]:
# Calculate percenteage values of deficit for primary and secondary
def prep_per_data(df):

    """ Get percentage values of deficit"""
    
    # Create dataframe and caluclate values
    df_def_per = pd.DataFrame()
    df_def_per['cat0'] = df.cat0
    df_def_per['cveg'] = round((1-(df.cveg_act / df.cveg_pot)) * 100).astype('int')
    df_def_per['soc'] = round((1-(df.soc_act / df.soc_pot)) * 100).astype('int')
    df_def_per['call'] = round((1-(df.call_act / df.call_pot)) * 100).astype('int')
    return df_def_per    


df_def_per_prim = prep_per_data(df_map_data_prim)
df_def_per_secd = prep_per_data(df_map_data_secd)

In [ ]:
# Get prepared driver bar data
df_bar = pd.read_csv(
    os.path.join(dir05, f'fig_ecozone/df_ecozone_bar_data_prim.csv'))

# Driver data from vegetation ecozones
df_bar_cveg = df_bar[df_bar.cat2 == 'cveg']
df_bar_cveg = df_bar_cveg \
    .assign(cat0 = pd.Categorical(
    df_bar_cveg['cat0'], ['trop1', 'trop2', 'subt', 'temp', 'bore', 'pola'])) \
    .sort_values('cat0') \
    .reset_index(drop=True)

# Driver data from soc ecozones
df_bar_soc = df_bar[df_bar.cat2 == 'soc']
df_bar_soc = df_bar_soc \
    .assign(cat0 = pd.Categorical(
    df_bar_soc['cat0'], ['trop1', 'trop2', 'subt', 'temp', 'bore', 'pola'])) \
    .sort_values('cat0') \
    .reset_index(drop=True)

# Driver data from vegetation global
df_bar_cveg_world = df_bar_cveg \
    .drop('cat0', axis=1) \
    .groupby('cat2') \
    .sum() \
    .reset_index(names='index')

# Driver data from soc global
df_bar_soc_world = df_bar_soc \
    .drop('cat0', axis=1) \
    .groupby('cat2') \
    .sum() \
    .reset_index(drop=True)

In [ ]:
# Get coastline shapefle
df_nearth = gpd.read_file(
    os.path.join(dir_nearth, 'ne_110m_coastline', 'ne_110m_coastline.shp'))

In [ ]:
# Ecozone shapefile
shp = gpd.read_file(os.path.join(dir_data, 'fao2010gez', 'gez_2010_wgs84.shp'))

# group regions
list_trop1 = [11]
list_trop2 = [12, 13, 14, 15, 16]
list_subt = [21, 22, 23, 24, 25]
list_temp = [31, 32, 33, 34, 35]
list_bore = [41, 42, 43]
list_pola = [50]

# Assign unique ecozone values and names
shp = shp.assign(code2 = np.nan)
shp.loc[shp.gez_code.isin(list_trop1), 'code2'] = 0
shp.loc[shp.gez_code.isin(list_trop2), 'code2'] = 1
shp.loc[shp.gez_code.isin(list_subt), 'code2'] = 2
shp.loc[shp.gez_code.isin(list_temp), 'code2'] = 3
shp.loc[shp.gez_code.isin(list_bore), 'code2'] = 4
shp.loc[shp.gez_code.isin(list_pola), 'code2'] = 5
shp.loc[shp.gez_code.isin(list_trop1), 'name'] = 'trop1'
shp.loc[shp.gez_code.isin(list_trop2), 'name'] = 'trop2'
shp.loc[shp.gez_code.isin(list_subt), 'name'] = 'subt'
shp.loc[shp.gez_code.isin(list_temp), 'name'] = 'temp'
shp.loc[shp.gez_code.isin(list_bore), 'name'] = 'bore'
shp.loc[shp.gez_code.isin(list_pola), 'name'] = 'pola'

# Merge/disolve polygons
shp2 = shp \
    .dissolve('code2') \
    .reset_index() \
    .drop(['gez_name', 'gez_code', 'gez_abbrev'], axis=1)

In [ ]:
# Colors
from palettable.cartocolors.qualitative import Safe_10
cpal = Safe_10

colors = [cpal.mpl_colors[i] for i in np.arange(0, len(cpal.colors))]

dict_colors = dict(urban = colors[0], 
                   pasture = colors[1],
                   crop = colors[2],
                   foresth = colors[3],
                   ambig = colors[4])

df_colors = pd.DataFrame() \
    .assign(keys=dict_colors.keys(),
            colors=dict_colors.values(),
            labels=['Urban', 'Pasture', 'Cropland', 'Managed forest', 
                    'Ambiguous'])

df_colors = df_colors.set_index('keys')
df_colors = df_colors[~df_colors.index.isin(['forestn', 'shrub', 'other'])] \
    .reset_index()

# colors for vegetation and soil carbon
# selected with https://color.adobe.com/ and https://projects.susielu.com/viz-palette
c_cveg, c_soc = '#73bd83', '#8f6358' 

In [ ]:
# Upper part of plot
def subplot_upper(ax, cat, title):
    """Bar plots of upper part of figure"""
    # category
    if cat == 'world':
        df_prim = df_map_data_prim[df_map_data_prim.cat0 == 'world']
        df_secd = df_map_data_secd[df_map_data_secd.cat0 == 'world']
        ymax = 840
    else:
        df_prim = df_map_data_prim[df_map_data_prim.cat0 != 'world']
        df_secd = df_map_data_secd[df_map_data_secd.cat0 != 'world']
        ymax=265

    # bars
    ax.barh(0.25, df_prim[df_prim.cat0 == cat].soc_pot, 
            color=c_soc, height=0.5, hatch='/////')
    ax.barh(-0.25, df_secd[df_secd.cat0 == cat].soc_pot, 
            color=c_soc, height=0.5, hatch='/////')
    ax.barh(0, df_prim[df_prim.cat0 == cat].soc_act, 
            color=c_soc, height=1)
    ax.barh(1.25, df_prim[df_prim.cat0 == cat].cveg_pot, 
            color=c_cveg, height=0.5, hatch='/////')
    ax.barh(0.75, df_secd[df_secd.cat0 == cat].cveg_pot, 
            color=c_cveg, height=0.5, hatch='/////')
    ax.barh(1, df_prim[df_prim.cat0 == cat].cveg_act, 
            color=c_cveg, height=1)

    # plot settings
    if cat == 'world':
        linewidth = 2
    if cat != 'world':
        linewidth = 1
    ax.set_ylim(-0.5, 1.6)
    ax.set_xlim(0, ymax)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(linewidth)
    ax.spines['bottom'].set_linewidth(linewidth)
    ax.xaxis.set_tick_params(width=linewidth)
    ax.yaxis.set_tick_params(width=linewidth)
    ax.set_yticks([])
    ax.patch.set_alpha(0)
    
    if cat == 'world':
        ax.set_title(title, fontsize='medium', loc='left', weight='bold')
        ax.set_xlabel('C stocks (PgC)', fontsize='small') 
        ax.tick_params(axis='x', labelsize='small')
        ax.tick_params(left = False, bottom = True)

    else:
        ax.set_title(title, fontsize='small', loc='left', weight='bold', pad=3)
        ax.set_xlabel(None)
        ax.set_xticks([0, 100, 200])
        ax.tick_params(axis='x', labelsize='small', length=2, pad=2)
        
    def plot_per_above_bars():
        if cat == 'world':
            y_cveg = df_prim[df_prim.cat0 == cat].cveg_pot.item() + 15
            y_soc = df_prim[df_prim.cat0 == cat].soc_pot.item() + 15
            s = 'small'

        if cat != 'world':
            y_cveg = df_prim[df_prim.cat0 == cat].cveg_pot.item() + 7.5
            y_soc = df_prim[df_prim.cat0 == cat].soc_pot.item() + 7.5
            s = 'small'
        
        ax.text(y_cveg, 1, 
                    str(int(df_def_per_secd[df_def_per_secd.cat0 == cat].cveg.item())) + '/' + 
                    str(int(df_def_per_prim[df_def_per_prim.cat0 == cat].cveg.item())) + '%',
                   va='center', size=s)
        ax.text(y_soc, 0,  
                    str(int(df_def_per_secd[df_def_per_secd.cat0 == cat].soc.item())) + '/' + 
                    str(int(df_def_per_prim[df_def_per_prim.cat0 == cat].soc.item()))  + '%',
                   va='center', size=s)
    
    plot_per_above_bars()

In [ ]:
# Lower part of plot
def subplot_lower(ax, df, ymax, a):
    """Barplots of lower part of figure"""
    #categories
    list_var = ['ambig', 'foresth', 'crop', 'pasture', 'urban']
    # sum classes for plotting (bars are plottet on top of each other)
    # urban
    df_sum = df[list_var].sum(axis=1)
    ax.bar(df.index + a, df_sum, color=dict_colors[list_var[-1]], width=0.3)
    # pasture
    df_sum = df[list_var[:-1]].sum(axis=1)
    ax.bar(df.index + a, df_sum, color=dict_colors[list_var[-2]], width=0.3)
    # crop
    df_sum = df[list_var[:-2]].sum(axis=1)
    ax.bar(df.index + a, df_sum, color=dict_colors[list_var[-3]], width=0.3)
    # foresth
    df_sum = df[list_var[:-3]].sum(axis=1)
    ax.bar(df.index + a , df_sum, color=dict_colors[list_var[-4]], width=0.3)
    # ambiguos
    df_sum = df[list_var[:-4]].sum(axis=1)
    ax.bar(df.index + a, df_sum, color=dict_colors[list_var[-5]], width=0.3)
    # plot 
    linewidth = 2
    ax.set_xlabel(None) 
    ax.spines['right'].set_visible(False) # remove right plot boundary
    ax.spines['top'].set_visible(False) # remove top plot boundary
    ax.spines['left'].set_linewidth(linewidth) # make left axis thicker
    ax.spines['bottom'].set_linewidth(linewidth) # make bottom axis thicker
    ax.xaxis.set_tick_params(width=linewidth) # make x-axis tick marks thicker
    ax.yaxis.set_tick_params(width=linewidth) # make y-axis tick marks thicker

In [ ]:
%%time
# Plot
fig = plt.figure(figsize=(9, 7), dpi=600)

gs = fig.add_gridspec(2, 3, 
                      hspace=0.1, 
                      wspace=0.3,
                      height_ratios=(0.66, 0.33),
                      width_ratios=(0, 0.15, 0.85))
# Axis upper plot
ax0 = fig.add_subplot(gs[0,:], zorder=2)
ax01 =fig.add_axes([0.33,.44,.43,.35], 
                   projection=ccrs.Robinson(central_longitude=10), 
                   zorder=1)

ax02 =fig.add_axes([0.16,.745,.1*1.8,.05*1.8], zorder=3)
ax03 =fig.add_axes([0.21,.55,.1,.05], zorder=3) # tropical rain
ax04 =fig.add_axes([0.31,.43,.1,.05], zorder=3) # tropical other
ax05 =fig.add_axes([0.51,.43,.1,.05], zorder=3) # subtropical
ax06 =fig.add_axes([0.71,.43,.1,.05], zorder=3) # temperate
ax07 =fig.add_axes([0.77,.63,.1,.05], zorder=3) # boreal
ax08 =fig.add_axes([0.66,.78,.1,.05], zorder=3) # polar

# Axis lower plot
ax10 = fig.add_subplot(gs[1,1])
ax11 = fig.add_subplot(gs[1,2])

# World map
df_nearth.plot(ax=ax01, transform=ccrs.PlateCarree(), 
               edgecolor = 'black', linewidth=0.5, 
               rasterized=True)
shp2.plot(ax=ax01, column='code2', transform=ccrs.PlateCarree(), cmap='Greys',
          rasterized=True)

ax01.set_extent((-140, 180, -60, 90), ccrs.PlateCarree())
ax01.axis('off')

# Bar plots upper
subplot_upper(ax02, 'world', 'Global')
subplot_upper(ax03, 'trop1', 'Tropical Rainforest')
subplot_upper(ax04, 'trop2', 'Other Tropical')
subplot_upper(ax05, 'subt', 'Subtropical')
subplot_upper(ax06, 'temp', 'Temperate')
subplot_upper(ax07, 'bore', 'Boreal')
subplot_upper(ax08, 'pola', 'Polar')

# Add description to world bar
ax02.text(550, 1.7, 'Deficit', ha='center', va='center', size='small', 
          bbox=dict(facecolor='#ffffff', edgecolor='None', pad=0.3, alpha=0))
ax02.text(25, 1, 'AGBC\nBGBC', ha='left', va='center', size='small')
ax02.text(200, 1, '+', ha='left', va='center', size='small')
ax02.text(25, 0, 'SOC', ha='left', va='center', size='small')

ax02.text(df_map_data_prim[df_map_data_prim.cat0 == 'world'].cveg_act.item() - 60, 1.25, 
          'P', ha='left', va='center', size='small')
ax02.text(df_map_data_prim[df_map_data_prim.cat0 == 'world'].cveg_act.item() - 60, 0.75, 
          'H', ha='left', va='center', size='small')
ax02.text(df_map_data_prim[df_map_data_prim.cat0 == 'world'].soc_act.item() - 60, 0.25, 
          'P', ha='left', va='center', size='small')
ax02.text(df_map_data_prim[df_map_data_prim.cat0 == 'world'].soc_act.item() - 60, -0.25, 
          'H', ha='left', va='center', size='small')

# Plot settings upper
ax0.set_xlim(-10, 10)
ax0.set_ylim(-10, 10)
ax0.axis('off')

# Lower plot
# World
subplot_lower(ax10, df_bar_cveg_world, 300, -0.15)
subplot_lower(ax10, df_bar_soc_world, 300, 0.15)
# Plot settings world
ax10.set_xlim(-0.5, 0.5)
ax10.set_ylim(0, 280)
ax10.plot([-0.1, 0.07], [210, 210], '#000000', linewidth=1)
ax10.plot([0.15, 0.15], [75, 98], '#000000', linewidth=1)
ax10.set_xticks([0])
ax10.set_xticklabels(['Global'], rotation=0, size='small')
# Add description to world plot
ax10.text(0.1, 209, 'AGBC\nBGBC', rotation=0, ha='left', va='center', size='small')
ax10.text(0.025, 108, 'SOC', rotation=0, ha='left', va='center', size='small')
ax10.set_ylabel('C deficit (PgC)', size='small')
ax10.tick_params(left = True, bottom = False, labelsize='small')

# Ecozones
subplot_lower(ax11, df_bar_cveg, 105, -0.15)
subplot_lower(ax11, df_bar_soc, 105, 0.15)

# Plot settings
ax11.set_xticks([0, 1, 2, 3, 4, 5])
ax11.set_xticklabels(['Tropical\nRainforest', 'Other\nTropical',
                     'Sub-\ntropical', 'Temp-\nerate', 'Boreal', 'Polar'], 
                     rotation=0, size='small')
ax11.tick_params(left = True, bottom = False, labelsize='small')
ax11.set_ylim(0, 85)
ax11.set_ylabel(None)

# Legend lower plot
legend_elements_lower = [Patch(facecolor=i, edgecolor=i, label=ii) for i,ii in 
                   zip(df_colors.colors, df_colors.labels)]
ax11.legend(handles=legend_elements_lower, loc='upper right', frameon=False, prop={'size': 'small'})

# Plot settings lower
for i in [ax0, ax01]:
    i.get_yaxis().set_visible(False)
    i.get_xaxis().set_visible(False)
    
# Plot settings
plt.figtext(0.12, 0.85, 'A', fontsize='x-large', weight='bold')
plt.figtext(0.12, 0.35, 'B', fontsize='x-large', weight='bold')

# Lines connecting world map and bar plots
ax0.plot([-5.25, -1.955], [-3.525, -2.25], '#000000', linewidth=1, linestyle=':')
ax0.plot([-2.575, 0.97], [-8.3, -3.035], '#000000', linewidth=1, linestyle=':')
ax0.plot([2.52, 2.25], [-8.3, 0.8], '#000000', linewidth=1, linestyle=':')
ax0.plot([5.075, 3.2], [-6.25, 1.70], '#000000', linewidth=1, linestyle=':')
ax0.plot([6.6, 3.8], [1.87, 2.95], '#000000', linewidth=1, linestyle=':')
ax0.plot([3.75, 2.65], [7.9, 4], '#000000', linewidth=1, linestyle=':');

# Export
plt.savefig(os.path.join(dir06, f'fig03_ecozone.pdf'), dpi=600, bbox_inches='tight', pad_inches=0.1)
plt.savefig(os.path.join(dir06, f'fig03_ecozone.png'), dpi=600, bbox_inches='tight', pad_inches=0.1)

---

## Plot Graphical Abstract

In [ ]:
# Define barplots of graphical abstract plot
def subplot_abstract2(ax, cat, title):

    """Bar plots of graphical abstract"""
    
    # Category
    if cat == 'world':
        df_prim = df_map_data_prim[df_map_data_prim.cat0 == 'world']
        ymax = 840
    else:
        df_prim = df_map_data_prim[df_map_data_prim.cat0 != 'world']
        ymax=265
        
    # Bars
    ax.barh(0.25, df_prim[df_prim.cat0 == cat].cveg_pot, 
            color=c_cveg, height=0.5, hatch='/////')
    ax.barh(0.25, df_prim[df_prim.cat0 == cat].cveg_act, 
            color=c_cveg, height=0.5)
    ax.barh(-0.25, df_prim[df_prim.cat0 == cat].soc_pot, 
            color=c_soc, height=0.5, hatch='/////')
    ax.barh(-0.25, df_prim[df_prim.cat0 == cat].soc_act, 
            color=c_soc, height=0.5)
    
    # Plot settings
    lw = 1.5
    ax.set_ylim(-0.5, 0.5)
    ax.set_xlim(0, ymax)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['bottom'].set_linewidth(lw)
    ax.xaxis.set_tick_params(width=lw)
    ax.yaxis.set_tick_params(width=lw)
    ax.set_yticks([])
    ax.patch.set_alpha(0)
    
    if cat == 'world':
        ax.set_title(title, fontsize=8, loc='left', weight='bold')
        ax.set_xlabel('C stocks (PgC)', fontsize=8) 
        ax.tick_params(axis='x', labelsize=8)
        ax.tick_params(left = False, bottom = True)

    else:
        ax.set_title(title, fontsize=8, loc='left', weight='bold', pad=3)
        ax.set_xlabel(None)
        ax.set_xticks([0, 100, 200])
        ax.tick_params(axis='x', labelsize=8, length=2, pad=2)
    
    def plot_per_above_bars():
        if cat == 'world':
            y_cveg = df_prim[df_prim.cat0 == cat].cveg_pot.item() + 15
            y_soc = df_prim[df_prim.cat0 == cat].soc_pot.item() + 15
            s = 'small'

        if cat != 'world':
            y_cveg = df_prim[df_prim.cat0 == cat].cveg_pot.item() + 7.5
            y_soc = df_prim[df_prim.cat0 == cat].soc_pot.item() + 7.5
            s = 'x-small'

        if cat == 'world':
            ax.text(y_cveg+20, 0.25, 
                    f'{df_def_per_prim[df_def_per_prim.cat0 == cat].cveg.item()}% '
                    'of potential carbon lost\ndue to land-use change', #'% of potential carbon lost',
                    va='center', size=8)
        else:
            ax.text(y_cveg, 0.25, 
                    str(int(df_def_per_prim[df_def_per_prim.cat0 == cat].cveg.item())) + '%',
                    va='center', size=8)
        ax.text(y_soc, -0.25,
                str(int(df_def_per_prim[df_def_per_prim.cat0 == cat].soc.item()))  + '%',
                va='center', size=8)
    
    plot_per_above_bars()

In [ ]:
# Plot
fig = plt.figure(figsize=(4, 4), dpi=300)  # Adjust height for upper plot only

gs = fig.add_gridspec(1, 1)

# Axis for the upper plot
ax0 = fig.add_subplot(gs[0,:], zorder=2)
ax01 = fig.add_axes([0.26, .25, .65, .4], 
                    projection=ccrs.Robinson(central_longitude=10), zorder=1)

ax02 = fig.add_axes([0.05, .75, .18*2.0, .09*1.8], zorder=3)
ax03 = fig.add_axes([0.05, .32, .18, .09], zorder=3)  # Tropical rain
ax04 = fig.add_axes([0.13, .12, .18, .09], zorder=3)  # Tropical other
ax05 = fig.add_axes([0.43, .12, .18, .09], zorder=3)  # Subtropical
ax06 = fig.add_axes([0.71, .12, .18, .09], zorder=3)  # Temperate
ax07 = fig.add_axes([0.71, .68, .18, .09], zorder=3)  # Boreal
ax08 = fig.add_axes([0.47, .68, .18, .09], zorder=3)  # Polar

# World map
df_nearth.plot(ax=ax01, transform=ccrs.PlateCarree(), 
               edgecolor='black', linewidth=0.5,
               rasterized=True)
shp2.plot(ax=ax01, column='code2', transform=ccrs.PlateCarree(), cmap='Greys',
          rasterized=True)

ax01.set_extent((-140, 180, -60, 90), ccrs.PlateCarree())
ax01.axis('off')

# Bar plots upper
subplot_abstract2(ax02, 'world', 'Global')
subplot_abstract2(ax03, 'trop1', 'Tropical Rainforest')
subplot_abstract2(ax04, 'trop2', 'Other Tropical')
subplot_abstract2(ax05, 'subt', 'Subtropical')
subplot_abstract2(ax06, 'temp', 'Temperate')
subplot_abstract2(ax07, 'bore', 'Boreal')
subplot_abstract2(ax08, 'pola', 'Polar')

# Description for world bar
ax02.text(770, 0.65, r'Deficit = Potential $-$ Actual', ha='left', va='center', size=8,
          fontweight='bold', bbox=dict(facecolor='#ffffff', edgecolor='None', pad=0.3, alpha=0))
ax02.text(15, 0.25, 'Vegetation', ha='left', va='center', size=8, fontweight='bold') # 
ax02.text(15, -0.25, 'Soil', ha='left', va='center', size=8, fontweight='bold')

ax02.annotate('', xy=(650, 0.425), xytext=(760, 0.65), va='center', 
            arrowprops=dict(facecolor='black', 
                            width=0.5,
                            headwidth=4,
                            headlength=5))

# Hide axis
ax0.set_xlim(-10, 10)
ax0.set_ylim(-10, 10)
ax0.axis('off')

# Connecting lines between world map and bar plots
ax0.plot([-7.15, -2.3], [-4.5, -2.1], '#000000', lw=1, ls=':') # Tropical rain
ax0.plot([-5.1, 1.4], [-9.65, -0.6], '#000000', lw=1, ls=':') # Tropical other
ax0.plot([2.55, 2.9], [-9.6, 0.3], '#000000', lw=1, ls=':') # Subtropical
ax0.plot([5.07, 3.4], [-7.25, 1.3], '#000000', lw=1, ls=':')  # Temperate
ax0.plot([2.1, 5.2], [1.9, 4.8], '#000000', lw=1, ls=':')  # Boreal
ax0.plot([-3.6, -1.15], [2.1, 4.65], '#000000', lw=1, ls=':')  # Polar

# export
plt.savefig(os.path.join(dir06, 'pdf/figx_graphical_abstract_new.pdf'), dpi=300, bbox_inches='tight')
plt.savefig(os.path.join(dir06, 'png/figx_graphical_abstract_new.png'), dpi=300, bbox_inches='tight');